# QPM : Assignement 6

### Librairies

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Data preparation

In [7]:
def annual_mean_return(df) :
    return df.mean() * 12

def annual_volatility(df) :
    return df.std() * np.sqrt(12)

def sharpe_ratio(df) :
    return annual_mean_return(df) / annual_volatility(df)


def get_multi_timeseries(list_underlying,startd,endd):
    '''
Function get_multi_timeseries:

Returns the full time series of selected stocks metrics (Open, High, Low, Close, Adj Close, Volume, etc.)
for a list of given stock tickers, over a specified time period and interval.

Inputs:
    - list_underlying: list of str ; tickers of the desired stocks.
    - startd / endd: str ; start and end dates (inclusive) defining the time range to retrieve, in format "YYYY-MM-DD".
    - metric: str or list-str ; name of the stock's metric(s) we want to select (close, open, high, ...).

Output:
    - DataFrame with:
        - index: pandas Timestamps (dates in ascending order),
        - columns: MultiIndex with first level = metrics (e.g., "Close"), 
                   second level = stock tickers.'''
    return yf.download(list_underlying,start=startd,end=endd)

In [8]:
period_value = "max"
interval_value = "1d"

In [9]:
tickers = ["AAPL","MSFT","AMZN","NVDA","TSLA","META"]
start_date = "2015-01-01"
end_date = "2022-12-31"
raw_market_data=get_multi_timeseries(tickers,start_date,end_date)
close_price_data = raw_market_data.xs("Close",axis = 1)
#Change the index type from Timestamp to DateTimme
close_price_data.index = pd.to_datetime(close_price_data.index, format = '%Y/%m/%d')

/var/folders/lr/7kzpljcn0zl52pt8zpscgfj40000gn/T/ipykernel_3502/59220051.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(list_underlying,start=startd,end=endd)
[*********************100%***********************]  6 of 6 completed


In [10]:
index_weighting=[0.0710,0.0651,0.0324,0.0284,0.0187,0.0184] #Given

In [11]:
#Transform the daily prices into monthly returns
monthly_returns = close_price_data.resample('M').ffill().pct_change()

In [12]:
#Compute excess returns assuming the riosk-free return is 0.
excess_returns = monthly_returns - 0

# Questions for Assignment 6

### Q6.1 Based on the sample data, compute the Markowitz portfolio weights

In [13]:
def portfolio_variance(weights, cov_matrix):
    """
    Computes the variance of a portfolio given asset weights and the covariance matrix.

    Parameters:
        weights (array-like): Portfolio weights.
        cov_matrix (DataFrame or ndarray): Covariance matrix of asset returns.

    Returns:
        float: Portfolio variance (not annualized here).
    """
    return weights.T @ cov_matrix @ weights

def markowitz_weights(excess_returns):
    """
    Computes the Markowitz minimum variance portfolio weights using excess returns.
    The optimization minimizes portfolio variance under the constraint that the weights sum to 1.
    
    Parameters:
        excess_returns (DataFrame): Excess returns (i.e., returns above the risk-free rate).

    Returns:
        ndarray: Optimal portfolio weights (sum to 1).
    """
    # Step 1: Compute the covariance matrix of excess returns
    cov_matrix = excess_returns.cov()
    # Step 2: Compute expected excess returns (not used in this version, but available)
    expected_returns = excess_returns.mean()
    # Step 3: Define constraint: weights must sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # Step 4: Initial guess — equal weighting
    init_guess = np.ones(len(expected_returns)) / len(expected_returns)
    # Step 5: Optimization — minimize portfolio variance subject to constraint
    opt_result = minimize(portfolio_variance, init_guess, args=(cov_matrix,), 
                      method='SLSQP', constraints=constraints)
    # Step 6: Extract and return the optimal weights
    markowitz_weights = opt_result.x
    return markowitz_weights

In [14]:
markowitz_weights(excess_returns)

array([ 0.24986716, -0.01885959,  0.15487716,  0.80815194, -0.1301815 ,
       -0.06385516])

### Q6.2 Then, using the market-capitalization weights, obtain the CAPM-implied expected returns.


The CAPM-implied expected returns are given by:

$$
\mu_{\text{CAPM}} = \Sigma \cdot w
$$

where:

- $\mu_{\text{CAPM}}$ is the vector of implied expected excess returns under CAPM,  
- $\Sigma$ is the covariance matrix of asset excess returns,  
- $w$ is the vector of market (index) portfolio weights.

This corresponds to the idea that each asset's expected return is proportional to its **covariance with the market**, under the assumption that the market portfolio is mean-variance efficient.

In [21]:
#Compute the returns covariance matrix
cov_matrix = excess_returns.cov()
#Compute the CAPM implied returns
capm_returns = cov_matrix @ np.array(index_weighting)
print(capm_returns)

Ticker
AAPL    0.001248
AMZN    0.001345
META    0.001034
MSFT    0.000881
NVDA    0.001646
TSLA    0.002079
dtype: float64


### Q6.3 Then, specify the pick matrix P and the view vector q that captures the following views for each of the assets:
#### AAPL: its absolute excess return is expected to be 10% per year.
### MSFT: its absolute excess return is expected to be 5% per year.
#### AMZN: no views
#### NVDA will outperform TSLA by 2% per year.
#### TSLA will underperform META by 1% per year.

In [22]:
#Pick matrix
P = np.array([
    [1, 0, 0, 0, 0, 0],   # View on AAPL
    [0, 0, 0, 1, 0, 0],   # View on MSFT
    [0, 0, 0, 0, 1, -1],  # NVDA outperforms TSLA
    [0, 0, -1, 0, 0, 1]   # TSLA underperforms META
])
#View vector.
q = np.array([0.10, 0.05, 0.02, -0.01])
Omega = 0.05*P@cov_matrix@P.T

### Q6.4 Use these views to compute the conditional expected excess return and conditional covariance matrix of excess returns μBL and ΣBL.

The Black-Litterman model adjusts expected returns and their uncertainty by blending market equilibrium (e.g., CAPM-implied) returns with investor views.

---

### 📌 Black-Litterman Expected Returns (Conditional Mean)

The adjusted expected returns under the Black-Litterman model are given by:

$$
\mu_{\text{BL}} = \mu_{\text{CAPM}} = \mathbb{E}[R \mid q]
$$

where:

- $\mu_{\text{CAPM}}$ is the equilibrium (prior) return vector (e.g., from CAPM),
- $q$ represents the investor’s views (mean of the subjective distribution),
- $\mathbb{E}[R \mid q]$ is the posterior mean return vector after incorporating views.

---

### 📌 Black-Litterman Posterior Covariance Matrix

The posterior covariance matrix under the Black-Litterman model is:

$$
\Sigma_{\text{BL}} = (1 + \tau) \Sigma - \tau^2 \Sigma P^\top \left( P \tau \Sigma P^\top + \Omega \right)^{-1} P \Sigma
$$

where:

- $\Sigma$ is the covariance matrix of returns,
- $\tau$ is a scalar representing the uncertainty in the prior estimates (typically small, e.g., 0.05),
- $P$ is the matrix identifying which assets the views refer to,
- $\Omega$ is the covariance matrix of the views,
- $\Sigma_{\text{BL}} = \text{Var}[R \mid q]$ is the posterior covariance matrix after views.

---

This model balances prior beliefs with investor confidence in new information.

In [18]:
# Black-Litterman Expected Return
tau = 0.05
Sigma=cov_matrix
mu_BL =capm_returns # Conditional expected return (E[R|q])

# Calculate V[R|q]
middle_term = np.linalg.inv(P @ (tau * Sigma) @ P.T + Omega)
V_BL = (1 + tau) * Sigma - (tau * Sigma @ P.T) @ middle_term @ (P @ (tau * Sigma)) #V[R|q]

### Q6.5 Use μBL and ΣBL to compute the mean-variance weights and compare them with the weights from the CAPM and the weights based on sample moments.

In [19]:
# Calculate portfolio weights w_BL
w_BL = np.linalg.inv(V_BL) @ mu_BL

In [20]:
comparison = index_weighting - w_BL

print(comparison)

[ 0.00071638  0.0031      0.00158473 -0.00153546  0.00103573  0.00068906]


#### We notice that the Black Litterman weights are very similar to the index weights